# Quickstart example for `ContextCite`

In this notebook, we'll overview the ContextCite API by going through a simple example.

You can also open the same notebook in [Google Colab](TODO)!

In [1]:
from context_cite import ContextCiter

[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/xfs/home/krisgrg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We'll use a tinyllama chat model for this example.

In [2]:
model_name_or_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

context = """
Attention Is All You Need

Abstract
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of the previous hidden state ht-1 and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples. Recent work has achieved significant improvements in computational efficiency through factorization tricks [21] and conditional computation [32], while also improving model performance in case of the latter. The fundamental constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.
"""
query = "What type of GPUs did the authors use in this paper?"

### The `ContextCiter` class

We can directly instantiate the `ContextCiter` class with a huggingface-style `pretrained_model_name_or_path`, together with a `context`, and a `query` (passed in as strings).

In [3]:
cc = ContextCiter.from_pretrained(model_name_or_path, context, query)

Alternatively, we can pass in a `model` and a `tokenizer`, which are instantiated from the `huggingface` library:

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.to("cuda")
cc = ContextCiter(model, tokenizer, context, query)

The `response` property of the ContextCiter class contains the response generated by the model. It is lazily generated when you access it.

In [5]:
cc.response

'The authors used eight P100 GPUs in their Transformer architecture for training on the WMT 2014 English-to-German translation task.'

Under the hood, the `ContextCiter` class applies a chat template to the
tokenized context and query, and then uses the model to generate a response.
That response is then stored in the `response` property.

### Attributing the response to sources within the context

To attribute the entire response and present the attributions in a human-readable format, we can use the `get_attributions` method, and pass in `as_dataframe=True`, as well as `top_k` to limit the number of sources to include in the attributions.

In [6]:
results = cc.get_attributions(as_dataframe=True, top_k=5)
results

  0%|          | 0/64 [00:00<?, ?it/s]

,Score,Source
0,13.837,The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.
1,12.590,"Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU."
2,5.306,"On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature."
3,1.898,"We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely."
4,1.525,"In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output."


`results` is a pandas styler object; to access the underlying dataframe:

In [7]:
results.data

,Score,Source
0,13.836627,The Transformer allows for significantly more ...
1,12.589814,Our model achieves 28.4 BLEU on the WMT 2014 E...
2,5.305615,On the WMT 2014 English-to-French translation ...
3,1.898218,"We propose a new simple network architecture, ..."
4,1.525449,"In this work we propose the Transformer, a mod..."


Alternatively, `.get_attributions()` can return the attribution scores as a `numpy` array, where the `i`th entry corresponds to the attribution score for the `i`th source in the context.

In [8]:
raw_results = cc.get_attributions()
raw_results

array([-0.        , -0.        ,  1.17331402, -0.        ,  1.89821833,
       -0.45667885, 12.58981409,  5.30561472,  0.        ,  0.        ,
       -0.        , -0.4833539 , -0.2234994 ,  0.53900443, -0.30795449,
        0.        ,  1.06010566, -0.        ,  1.52544853, 13.83662735])

### Attributing parts of the response

`.get_attributions()` optionally takes in `start_idx` and `end_idx` to
attribute only a part of the response.

To make it easier to attribute parts of the response, the `ContextCiter` class
has a utility property `response_with_indices` that contains the response annotated with
the index of each word within the response. You can access this with
`cc.response_with_indices`.

In [9]:
print(cc.response_with_indices)

[0]The [4]authors [12]used [17]eight [23]P100 [28]GPUs [33]in [36]their [42]Transformer [54]architecture [67]for [71]training [80]on [83]the [87]WMT [91]2014 [96]English-to-German [114]translation [126]task[130].


For example, we can attribute a part of the response like so:

In [10]:
start, end = 17, 33
print(f"Response to attribute: \"{cc.response[start:end]}\"")
cc.get_attributions(start_idx=start, end_idx=end, as_dataframe=True, top_k=5)

Response to attribute: "eight P100 GPUs "


,Score,Source
0,12.770,The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.
1,2.424,"On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature."
2,0.303,"Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of the previous hidden state ht-1 and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples."
3,0.279,"The fundamental constraint of sequential computation, however, remains."
4,0.052,Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train.


In [11]:
start, end = 83, 130
print(f"Response to attribute: \"{cc.response[start:end]}\"")
cc.get_attributions(start_idx=start, end_idx=end, as_dataframe=True, top_k=5)

Response to attribute: "the WMT 2014 English-to-German translation task"


,Score,Source
0,12.400,"Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU."
1,0.643,"On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature."
2,0.231,1 Introduction
3,0.040,The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder.
4,0.011,"Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [2, 19]."
